# **Практика EDA-4**

Используя информацию по мидиям Петербурга и Магадана, которые представлены в виде двух массивов petersburg и magadan в разделе ANOVA-тест, проверьте данные на нормальность и на наличие корреляции. 

Для этого:

* Объедините два массива в DataFrame. 
* Обоснуйте выбор теста на корреляцию. 
* Сделайте вывод по гипотезе.
* Ответьте на вопрос: Какова разница в среднем размере мидии в зависимости от города-производителя
* Обоснуйте выбор теста. 
* Сделайте вывод по гипотезе.

***
### **Обработка Данных**

In [12]:
# Блок с вызовом библиотек
import pandas as pd #для работы с таблицами
from IPython.display import display
import numpy as np
from numpy.random import rand
import wget

# Тесты
from scipy.stats import shapiro # Тест Шапиро-Уилка
from scipy.stats import normaltest # Тест Д'Агостино
from scipy.stats import ttest_ind # Независимый Т-тест

# Корреляция
from scipy.stats import spearmanr

In [2]:
# Объявляем списком датасеты
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764,
           0.0689]

In [26]:
# Создаём датафреймы по городам из списков и объединяем в один
mussels1 = pd.DataFrame(np.array(petersburg), columns=['petersburg'])
mussels2 = pd.DataFrame(np.array(magadan), columns=['magadan'])

mussels=pd.concat([mussels1,mussels2], axis=1, ignore_index=False)
# Заполняем пропуск средним значением по столбцу
mussels = mussels.fillna({'petersburg':mussels['petersburg'].mean(), 'magadan':mussels['magadan'].mean()})
display(mussels)

,petersburg,magadan
0,0.097400,0.1033
1,0.135200,0.0915
2,0.081700,0.0781
3,0.101600,0.0685
4,0.096800,0.0677
5,0.106400,0.0697
6,0.105000,0.0764
7,0.103443,0.0689


***
### **Проверка на нормальность**

In [27]:
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'
# Уровень значимости - альфа:
alpha = 0.05

In [28]:
# Тест Шапиро-Уилка
_, p = shapiro(mussels)
print('p=%.3f' % p)

if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.100
Данные распределены нормально


Поскольку данные распределены по **нормальному** закону, выбираем для проверки статистической гипотезы тест из параметрической категории - **корреляция Пирсона**.

***
### **Тест на корреляцию**

In [49]:
# Выдвигаем гипотезы
H0 = 'Нет влияния размеров раковин мидий в разных городах друг на друга.'
Ha = 'Есть влияние размеров раковин мидий в разных городах друг на друга.'

In [47]:
# Кореляция Пирсона
mus_cor = mussels.corr()
pier_cor = mus_cor.iloc[0][1].round(3)
display(mus_cor)
display(f'Корреляция между признаками = {pier_cor}')

,petersburg,magadan
petersburg,1.000000,0.244558
magadan,0.244558,1.000000


'Корреляция между признаками = 0.245'

In [50]:
if p>alpha:
    print(f"{pier_cor} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
else:
    print(f"{pier_cor} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

0.245 <= 0.05. Мы отвергаем нулевую гипотезу. Есть влияние размеров раковин мидий в разных городах друг на друга.


Согласно таблице силы связи между признаками, прослеживается **очень слабая взаимосвязь**. Это значит, что влияние размеров раковин Петербуржских мидий на размер раковин мидий в Магадане присутствует в незначительной степени. Однако это влияние больше похоже на **ложную** взаимосвязь.

***
### **Какова разница в среднем размере мидии в зависимости от города-производителя**

Средний размер мидий - зависимая количественная переменная.  
Город-производитель - независимая категориальная переменная.  
Поскольку средний размер мидий берется для двух разных городов (разных совокупностей одной категории), мы выбираем независимый **t-тест**.

In [40]:
# Выдвигаем гипотезы
H0 = 'Нет значимой разницы между средними размерами раковины в разных городах.'
Ha = 'Есть значимая разница между средними размерами раковины в разных городах.'

In [44]:
# Проводим t-тест
print('\n' + "*** Результаты независимого T-теста ***")
test_results = ttest_ind(mussels['petersburg'], mussels['magadan'], equal_var=True)

p = round(test_results[1],4)

if p>alpha:
    print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
else:
    print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")


*** Результаты независимого T-теста ***
0.0027 <= 0.05. Мы отвергаем нулевую гипотезу. Есть значимая разница между средними размерами раковины в разных городах.


То, в каком городе выращивались мидии, **влияет** на её средний размер.